<a href="https://colab.research.google.com/github/dimsparagis0210/DFU-using-Deep-Learning/blob/main/EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EfficientNet Model Development

#### The goal of this BSc thesis is to develop a Deep Learning model for Object Detection. Specifically, the object of the model will be to detect ulcers in Diabetic Foots (DFU)

## Getting Dataset from Google drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Define dataset path (adjust if needed)
dataset = '/content/drive/MyDrive/DFU_Data_Tensorflow'
original = dataset + '/images'
trainset = dataset + '/train'
testset = dataset + '/test'
validset = dataset + '/valid'

# List categories (folders)
os.listdir('/content')

['.config', 'drive', 'my_dir', 'sample_data']

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import cv2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

## Data Preprocessing

### The dataset contains a csv with the annotations' coordinates

In [ ]:
import pandas as pd
import os

# Define paths to the annotation files
train_csv_path = trainset + '/_annotations.csv'
test_csv_path = testset + '/_annotations.csv'
valid_csv_path = validset + '/_annotations.csv'

# Load the CSV files into DataFrames
df_train = pd.read_csv(train_csv_path)
df_test = pd.read_csv(test_csv_path)
df_valid = pd.read_csv(valid_csv_path)

# Display the first few rows to understand the structure
print(df_train.head())

                                            filename  width  height  class  \
0  51814847_548907345629031_1716986440128135168_n...    640     640  Ulcer   
1  51814847_548907345629031_1716986440128135168_n...    640     640  Ulcer   
2  51814847_548907345629031_1716986440128135168_n...    640     640  Ulcer   
3     20_jpg.rf.2ae76d51fdf025da6450aaa95081e571.jpg    640     640     bb   
4  40152324_676265276071443_3891772576753516544_n...    640     640  Ulcer   

   xmin  ymin  xmax  ymax  
0    88   115   237   321  
1   382   157   523   407  
2   594   222   640   425  
3   255   190   387   269  
4   147   356   235   392  


### Data Augmentation

In [ ]:
def load_and_preprocess_image(image_path, bbox):
    # Read the image from disk
    image = tf.io.read_file(image_path)
    # Decode the image (assuming JPEG format)
    image = tf.image.decode_jpeg(image, channels=3)
    # Resize the image to the desired size
    image = tf.image.resize(image, [224, 224])
    # Normalize pixel values to the range [0, 1]
    image = image / 255.0
    # Convert bounding box coordinates to float32
    bbox = tf.cast(bbox, dtype=tf.float32)
    return image, bbox


### Create a new dataset in a format that Keras can accept

In [ ]:
def create_dataset(df, image_dir):
    # Extract image file paths and bounding boxes
    image_paths = [os.path.join(image_dir, fname) for fname in df['filename']]
    bboxes = df[['xmin', 'ymin', 'xmax', 'ymax']].values

    # Create a TensorFlow Dataset from the image paths and bounding boxes
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, bboxes))

    # Define a function to load images and parse annotations
    def load_image_and_bbox(image_path, bbox):
        image, bbox = load_and_preprocess_image(image_path, bbox)
        return image, bbox

    # Map the function over the dataset
    dataset = dataset.map(load_image_and_bbox, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

# Create the datasets
train_dataset = create_dataset(df_train, trainset)
test_dataset = create_dataset(df_test, testset)
valid_dataset = create_dataset(df_valid, validset)

### Fetching data in separate batches

In [ ]:
batch_size = 32
train_dataset = train_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = valid_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

## Model Development

### Hyperparameter Optimization (Keras-Tuner)

In [ ]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
import keras_tuner as kt

def build_model(hp):
    inputs = layers.Input(shape=(224, 224, 3))
    base_model = EfficientNetB0(include_top=False, input_tensor=inputs, weights='imagenet')
    base_model.trainable = hp.Boolean('trainable_base', default=False)

    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1))(x)
    x = layers.Dense(hp.Int('units', min_value=128, max_value=512, step=64), activation='relu')(x)
    outputs = layers.Dense(4, activation='sigmoid')(x)  # Adjust based on your output requirements

    model = models.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')),
        loss='mean_squared_error',  # Adjust based on your task
        metrics=['accuracy']  # Adjust based on your task
    )

    return model

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=2,
    directory='hyperparam_tuning',
    project_name='efficientnet_tuning'
)

Reloading Tuner from hyperparam_tuning/efficientnet_tuning/tuner0.json


In [ ]:
tuner.search(train_dataset, validation_data=val_dataset, epochs=10)

Trial 20 Complete [00h 05m 03s]
val_accuracy: 0.34415584802627563

Best val_accuracy So Far: 0.5844155550003052
Total elapsed time: 01h 04m 51s


In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)